# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-16 01:00:31] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-16 01:00:31] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-16 01:00:31] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-16 01:00:34] INFO server_args.py:1653: Attention backend not specified. Use fa3 backend by default.


[2026-01-16 01:00:34] INFO server_args.py:2552: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=49.54 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=44.97 GB):  25%|██▌       | 5/20 [00:00<00:01, 10.88it/s]

Capturing batches (bs=24 avail_mem=44.94 GB):  60%|██████    | 12/20 [00:00<00:00, 20.05it/s]

Capturing batches (bs=1 avail_mem=44.87 GB):  95%|█████████▌| 19/20 [00:01<00:00, 24.51it/s]

Capturing batches (bs=1 avail_mem=44.87 GB): 100%|██████████| 20/20 [00:01<00:00, 18.66it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Niek and I am a passionate IT and software developer with over ten years experience in the field. I started my career in 2005 as a professional accountant and I have spent my time working in a variety of roles, including project management and software development. After my career in IT, I've worked in various roles in the tech industry, including web development and mobile app development.
My skills include proficiency in C++, HTML, CSS, Java, JavaScript, and Python. I am also proficient in using Git, Docker, and AWS services. I'm currently learning data science and machine learning using Python. I have a passion for
Prompt: The president of the United States is
Generated text:  a person who has a very important role in the country. He/she is in charge of the country and makes sure everyone is safe. 

The president is always chosen by the people who vote. This is called the popular vote. But, this is not the only way to choose a president. An

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Skill/Ability] who has been [Reason for being] for [Number of Years]. I'm passionate about [What I love to do]. I'm [What I enjoy doing]. I'm [What I'm known for]. I'm [What I'm proud of]. I'm [What I hope to achieve]. I'm [What I'm looking forward to doing]. I'm [What I'm looking forward to doing]. I'm [What I'm looking forward to doing]. I'm [What I'm looking forward to doing].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French National Library, and the French Parliament House. Paris is a cultural and economic hub of the country and is a popular tourist destination. The city is also known for its cuisine, including French cuisine, and its annual festivals and events. Paris is a vibrant and dynamic city with a rich history and a diverse population. The city is home to many museums, art galleries, and cultural institutions, including the Louvre, the Musée d'Orsay

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes. This could lead to more sophisticated and adaptive AI systems that can learn from feedback and improve their performance over time.

2. Greater emphasis on ethical considerations: As AI becomes more integrated with human intelligence, there will be a greater emphasis on ethical considerations. This could lead to more stringent regulations and guidelines for AI development and deployment, as well as increased scrutiny of AI systems that are designed to be used in ways that could harm humans.





### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Sarah. I am a tenacious, driven, and passionate writer who thrives on challenging myself creatively. I have a strong work ethic and am always pushing myself to be the best. I have been living in the city for the past three years, and I love exploring new places and learning new things. I am a student at the University of the Arts, and I am constantly inspired by the work of other writers and artists. I love taking risks, experimenting with different mediums, and pushing the boundaries of what is possible. I am a mentor to many students, and I am always looking for new opportunities to inspire them. Thank you for

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

Paris, the 9th-largest city in the world by population, is the political, cultural, and economic capital of France, known for its rich history, 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 and

 loyalty

.

 These

 technologies

In [6]:
llm.shutdown()